In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import GRU, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
#from dask import dataframe as dsk

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [5]:
!wget 'https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0'

--2023-07-25 10:07:21--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2023-07-25 10:07:21--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce4b4578ac5c224ab9cc231047f.dl.dropboxusercontent.com/cd/0/inline/CAhIpzK5QNur8iMRAz3kPFNEFTy3v6oIAMsbGxRM-yelwUr0DtBvZw0w03TDdCrWcjAyTm8aSUnqn24ENbGfmT-1NDJpUzidiOkhxS5De2cPUtAI5UdEDWnUTP4ZFleJv5HbL30i4gwIrTMhTUOma1l5eYQ9UmiNY-UbLTa6e2g-xw/file# [following]
--2023-07-25 10:07:21--  https://uce4b4578ac5c224ab9cc231047f.dl.dropboxusercontent.com/cd/0/inline/CAhIpzK5QNur8iMRAz3kPFNEFTy3v6oIAMsbGxRM-yelwUr0DtBvZw0w03TDdCrWcj

In [6]:
!unzip stacksample.zip?dl=0

Archive:  stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [7]:
!rm stacksample.zip?dl=0

In [15]:
consideredMaxId = 1800000

In [16]:
listofQuestions = pd.read_csv('/content/Questions.csv', usecols=["Id", "Body"], encoding = "ISO-8859-1")

In [17]:
listofTags = pd.read_csv('/content/Tags.csv',usecols = ["Id", "Tag"], encoding = "ISO-8859-1")

In [18]:
listofQuestions = listofQuestions.loc[listofQuestions['Id'] < consideredMaxId]
listofTags = listofTags.loc[listofTags['Id'] < consideredMaxId]

In [19]:
def GetProperData(txt):
	soup = BeautifulSoup(txt,'lxml')
	paras=soup.find_all('p')
	paras=[p.text for p in paras]
	return paras

In [20]:
listofQuestions.sort_values(by=['Id'])
listofQuestions['Body'] = listofQuestions.apply(lambda x: GetProperData(x['Body']), axis=1)
listofQuestions['Body'] = listofQuestions.Body.apply(lambda x: ' '.join(x))

In [21]:
listofQuestions.head()

,Id,Body
0,80,I've written a database generation script in S...
1,90,Are there any really good tutorials explaining...
2,120,Has anyone got experience creating SQL-based A...
3,180,This is something I've pseudo-solved many time...
4,260,I have a little game written in C#. It uses a ...


In [22]:
listofQuestions._get_value(0, 'Body', takeable=False)

"I've written a database generation script in SQL and want to execute it in my Adobe AIR application: I execute this in Adobe AIR using the following methods: No errors are generated, however only tRole exists. It seems that it only looks at the first query (up to the semicolon- if I remove it, the query fails). Is there a way to call multiple queries in one statement?"

In [23]:
listofTags.isnull().sum()

Id      0
Tag    42
dtype: int64

In [24]:
listofTags["Tag"].fillna("Null_Val", inplace = True)

In [ ]:
listofTags.isnull().sum()

Id     0
Tag    0
dtype: int64

In [26]:
n = 10
topTag = listofTags['Tag'].value_counts()[:n].index.tolist()

In [27]:
tagList_top = listofTags[listofTags['Tag'].isin(topTag)]

In [28]:
listofTags = tagList_top

             Id         Tag
8           120     asp.net
14          260          c#
15          260        .net
18          330         c++
22          470        .net
...         ...         ...
103416  1799380         php
103420  1799620  javascript
103421  1799620      jquery
103433  1799820         c++
103436  1799990        java

[21746 rows x 2 columns]


In [29]:
df = listofQuestions
a = listofTags['Id']
df = df[df['Id'].isin(a)]
listofQuestions = df

In [30]:
def add_tags(question_id):
    return listofTags[listofTags['Id'] == question_id['Id']].Tag.values

temp = listofTags.apply(add_tags, axis=1)
print(type(temp))

<class 'pandas.core.series.Series'>


In [31]:
listofTags=pd.concat([listofTags, temp.rename('Tags')], axis=1)
listofTags.head()

,Id,Tag,Tags
8,120,asp.net,[asp.net]
14,260,c#,"[c#, .net]"
15,260,.net,"[c#, .net]"
18,330,c++,[c++]
22,470,.net,[.net]


In [32]:
listofTags = listofTags.drop(['Tag'], axis = 1)

In [33]:
listofTags = listofTags.drop_duplicates(subset="Id", keep="first", inplace=False)

In [35]:
listofQuestions = pd.merge(listofQuestions, listofTags, on='Id')

In [ ]:
listofQuestions.isnull().sum()

Id      0
Body    0
Tags    0
dtype: int64

In [36]:
sent_lens=[]
for sent in listofQuestions['Body']:
    sent_lens.append(len(word_tokenize(sent)))

In [37]:
max(sent_lens)

1819

In [38]:
for percentile in [0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97]:
  print('For percentile:', percentile, ":", np.quantile(sent_lens, percentile))


For percentile: 0.81 : 157.0
For percentile: 0.82 : 160.0
For percentile: 0.83 : 164.0
For percentile: 0.84 : 168.0
For percentile: 0.85 : 173.0
For percentile: 0.86 : 178.0
For percentile: 0.87 : 184.0
For percentile: 0.88 : 190.0
For percentile: 0.89 : 196.0
For percentile: 0.91 : 211.0
For percentile: 0.92 : 220.0
For percentile: 0.93 : 231.0
For percentile: 0.94 : 242.0
For percentile: 0.95 : 256.0
For percentile: 0.96 : 275.0
For percentile: 0.97 : 298.0


In [39]:
max_len = int(np.quantile(sent_lens, 0.88))
print(max_len)

190


In [41]:
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(listofQuestions['Body'])

In [44]:
multilabel = MultiLabelBinarizer()
multilabel.fit_transform(listofQuestions['Tags'])

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [45]:
multilabel.classes_

array(['.net', 'asp.net', 'c#', 'c++', 'iphone', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [46]:
Q_train, Q_test = train_test_split(listofQuestions,test_size=0.2,random_state=2)
x_train = Q_train['Body']
y_train = Q_train['Tags']
x_test  = Q_test['Body']
y_test  = Q_test['Tags']

In [47]:
#input to model Train_X = sequences_matrix_train
sequences_train = tok.texts_to_sequences(x_train)
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [48]:
#input to model Test_X = sequences_matrix_test
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen=max_len)

In [49]:
vocab_len = len(tok.index_word.keys())
vocab_len

39713

In [50]:
#Output to model Train_Y = y_train_multiLevel
y_train_multiLevel = multilabel.transform(y_train)

In [51]:
outputNode = len(multilabel.classes_)
max_len, outputNode

(190, 10)

In [52]:
#Output to model Test_Y = y_test_multiLevel
y_test_multiLevel = multilabel.transform(y_test)

In [53]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])

    layer = Embedding(vocab_len+1,1500,input_length=max_len,
                      mask_zero=True)(inputs)
    layer = GRU(400)(layer)
    layer = Dense(200,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dense(100,name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(outputNode,name='out_layer')(layer)
    #layer = Activation('sigmoid')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [54]:
model = RNN()
model.summary()

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 190)]             0         
                                                                 
 embedding (Embedding)       (None, 190, 1500)         59571000  
                                                                 
 gru (GRU)                   (None, 400)               2282400   
                                                                 
 FC1 (Dense)                 (None, 200)               80200     
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 FC2 (Dense)                 (None, 100)               20100     
                                                                 
 activation_1 (Activation)   (None, 100)               0     

In [ ]:
model.fit(sequences_matrix_train,y_train_multiLevel,batch_size=50,
          epochs=3,validation_data=(sequences_matrix_test,y_test_multiLevel))


Epoch 1/3
 77/237 [========>.....................] - ETA: 18:49 - loss: 0.4239 - accuracy: 0.1431

In [ ]:
predictions=model.predict(sequences_matrix_test)

from sklearn.metrics import roc_auc_score

roc_auc_score(y_test_multiLevel,predictions)